# **Import Data**

## input file Path below 

In [9]:
gift_file='Gift_data.csv'

In [10]:
import pandas as pd
gift = pd.read_csv(gift_file,low_memory=False)

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
gifttest=gift
len(gifttest)

519793

In [4]:
gifttest

,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,Fund Description,Appeal ID,Appeal Description,Luminate Online Gift ID,NetCommunity Online Gift ID,Gift Code,OPX Provider Description
0,Individual,1026027.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98KR,98 April Renewal,NaN,NaN,NaN,NaN
1,Individual,1041282.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
2,Individual,1041328.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98JR,98 March Renewal,NaN,NaN,NaN,NaN
3,Individual,1111161.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
4,Individual,1113516.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519788,Individual,1247705.0,6/30/2023,$60.00,$60.00,Cash,Workplace Giving,4009 PYDEDLCF,Payroll Deductions LCF - 4009,Payroll LCF Employee,Payroll Deductions - LCF Employees,LCFEMPGIV23,LCF Employee Giving Campaign FY23,NaN,NaN,NaN,NaN
519789,Individual,1049457.0,6/30/2023,$100.00,$100.00,Cash,Individual Giving,4001 INDGIV,Individual Giving - 4001,700 FUND,General Contribution,NOV23R,November 2022 Appeal,NaN,NaN,NaN,NaN
519790,Individual,1248910.0,6/30/2023,$45.00,$45.00,Cash,Workplace Giving,4009 PYDEDLCF,Payroll Deductions LCF - 4009,Payroll LCF Employee,Payroll Deductions - LCF Employees,LCFEMPGIV23,LCF Employee Giving Campaign FY23,NaN,NaN,NaN,NaN
519791,Organization,1269332.0,6/30/2023,$75.00,$75.00,MG Pledge,Individual Giving,4005 C&FCONT,Corporations & Foundations Contributions - 4005,700 FUND,General Contribution,JUN23R,June 2023 Appeal,NaN,NaN,NaN,NaN


# Cleaning: String Strip
- combine 'Appeal Description' and 'Campaign Description' to a new column called 'concatenated'
- If NAs present in the 'concatenated' column, fill in with values from 'Fund Description'
- strip string by removing punctuations, duplicate words, and unnecessary date info

In [5]:
#concate 'Appeal Description' compaign description
gifttest['concatenated']=gifttest['Appeal Description']+' '+gifttest['Campaign Description']
#fillna fund description
gifttest['concatenated'].fillna(gifttest['Fund Description'], inplace=True)
# gifttest = gifttest.dropna(subset=['concatenated'])
gifttest['concatenated']=gifttest['concatenated'].str.lower()
#strip
# Define regular expression patterns
pattern_numbers = r'\d+'
pattern_months = r'\b(?:january|february|march|april|may|june|july|august|september|october|november|december)\b'
pattern_punctuation = r'[^\w\s]'
pattern_duplicated_words = r'\b(\w+)\b(?=.*\b\1\b)'
def duplicate(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))
    unique_text = ' '.join(unique_words)
    return unique_text

gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_numbers, '', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_months, '', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_punctuation, ' ', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_duplicated_words, r'\1', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].apply(lambda x: duplicate(x))

In [6]:
len(gifttest['concatenated'])

519793

# EDA: Event Words

### Use Counter to see the top frequent words 

In [7]:
from collections import Counter
words = ' '.join(gifttest['concatenated']).split()
word_count = Counter(words)
common_keywords = [word for word, count in word_count.most_common() if count > 2]


#drop stopwords
word_count.most_common()[:20]

[('mail', 212132),
 ('direct', 211926),
 ('old', 181770),
 ('fy', 127893),
 ('giving', 90170),
 ('mo', 79146),
 ('x', 73194),
 ('renewal', 55380),
 ('general', 53055),
 ('annual', 52400),
 ('appeal', 48485),
 ('budget', 44705),
 ('operating', 43037),
 ('newsletter', 41724),
 ('of', 31315),
 ('circle', 27094),
 ('love', 26585),
 ('campaign', 24360),
 ('end', 21412),
 ('individual', 18763)]

In [9]:
import re
stopwordlist = ['fy', 'mo', 'x', 'end','old']

# Create a regular expression pattern to match stopwords
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in stopwordlist) + r')\b'

# Remove stopwords from the 'concatenated' column
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern, '', regex=True)
gifttest['concatenated'] 

0                                             direct mail 
1                                             direct mail 
2                                             direct mail 
3                                             direct mail 
4                                             direct mail 
                                ...                       
519788    lcf employee giving campaign  payroll deductions
519789                            appeal individual giving
519790    lcf employee giving campaign  payroll deductions
519791       appeal corporations foundations contributions
519792    lcf employee giving campaign  payroll deductions
Name: concatenated, Length: 519793, dtype: object

### Use Document-term matrix to see the top frequent words
- set ngram to be 2-4, since the event name may contain several words (one word doesn't make sense)
- find the combination with the highest occurrences: up to the top 250

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(ngram_range=(2, 4))
X = cv.fit_transform(gifttest['concatenated'])

# Convert the result to a DataFrame for analysis
count_matrix = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())
count_matrix[:10]

,abbott work,abbott work place,abbott work place giving,abbvie work,abbvie work place,abbvie work place giving,abcf campaign,abcf campaign acquisition,above general,above general contr,...,ypg food frenzy,ypg food frenzy general,ypg food frenzy special,ypg little,ypg little city,ypg little city on,zips acquisitions,zurich employee,zurich employee giving,zurich employee giving workplace
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
combination_counts = count_matrix.sum()

# Find the combination with the highest occurrences
max_combination = combination_counts.idxmax()
max_occurrences = combination_counts[max_combination]
print(combination_counts.sort_values(ascending=False)[:50])

direct mail                            211317
general operating                       43037
general operating budget                43035
operating budget                        43035
annual giving                           41202
of love                                 26582
circle of                               26582
circle of love                          26582
individual giving                       17322
employee giving                         17191
work place                              16083
special events                          13733
love annual                             11421
of love annual                          11421
of love annual giving                   11421
love annual giving                      11421
circle of love annual                   11421
year appeal                             11261
newsletter general                      10884
newsletter general operating            10884
newsletter general operating budget     10884
giving campaign                   

In [15]:
print(combination_counts.sort_values(ascending=False)[50:100])

recurring gift pledge payment        5917
gift pledge payment                  5917
recurring gift                       5917
gift pledge                          5917
mailing annual                       5896
mailing annual giving                5896
annual dinner                        5561
city invitational                    5494
little city invitational             5494
labels general                       5329
labels general operating budget      5329
labels general operating             5329
love general                         5237
love general operating               5237
of love general operating            5237
circle of love general               5237
of love general                      5237
love general operating budget        5237
work place giving                    5035
place giving                         5035
santa stickers                       5026
year appeal direct                   4840
year appeal direct mail              4840
address labels                    

In [17]:
print(combination_counts.sort_values(ascending=False)[100:150])

mail package                            4183
teddy bear direct mail                  4183
teddy bear direct                       4183
teddy bear                              4183
payroll deduction                       3965
smiling santa seals                     3862
seals low donors                        3862
santa seals                             3862
seals low                               3862
santa seals low                         3862
low donors                              3862
smiling santa seals low                 3862
santa seals low donors                  3862
smiling santa                           3862
parent monthly                          3859
letter general                          3840
letter general operating                3840
letter general operating budget         3840
giving pfg                              3835
family giving pfg                       3835
tournament special events               3820
invitational golf tournament special    3820
tournament

In [18]:
print(combination_counts.sort_values(ascending=False)[150:200])

annual dinner special events           3344
dinner special                         3344
annual dinner special                  3344
dinner special events                  3344
fall newsletter direct                 3306
fall newsletter direct mail            3306
regular donor renewal                  3304
reg regular                            3283
reg regular donor                      3283
reg regular donor renewal              3282
giving pfg annual                      3156
family giving pfg annual               3156
appeal annualgvgfy                     3142
winter newsletter general              3075
winter newsletter general operating    3075
ins octoberr                           3043
high donor                             3014
aug ren                                2946
teaser copy renewal                    2904
copy renewal                           2904
address labels regular donors          2882
labels regular                         2882
regular donors                  

In [21]:
print(combination_counts.sort_values(ascending=False)[200:250])

giving general                         2180
summer newsletter general operating    2171
summer newsletter general              2171
of illinois                            2146
illinois work                          2142
of illinois work place                 2142
of illinois work                       2142
illinois work place                    2142
summer newsletter direct mail          2140
summer newsletter direct               2140
illinois work place giving             2135
university of                          2131
university of illinois work            2131
university of illinois                 2131
monthly contributions                  2131
parent monthly contributions           2131
book sale                              2123
gifts general                          2087
gifts general contributions            2083
giving general operating budget        2073
giving general operating               2073
employee giving general                2072
employee giving general operatin

In [26]:
print(combination_counts.sort_values(ascending=False)[300:350])

spring newsletter direct mail              1412
spring newsletter direct                   1412
fall appeal                                1408
autism appeal                              1374
holiday appeal opening                     1364
opening doors                              1364
doors annual giving                        1364
doors annual                               1364
opening doors annual                       1364
opening doors annual giving                1364
holiday appeal opening doors               1364
appeal opening doors                       1364
appeal opening                             1364
appeal opening doors annual                1364
autism appeal annual                       1359
autism appeal annual giving                1359
poker tournament casino                    1328
tournament casino                          1328
poker tournament                           1328
poker tournament casino night              1328
casino night                            

### Manually create a new events_list 
- refer to events in orginal .csv file
- based on former top frequent events (eyeball)

Duplicate event names:

- general operating budget, annual giving, teddy bear, book sale, parent = annual giving
- direct mail, circle of love, newsletter, pledge, santa stickers, smiles campaign, holiday mailing, verne carson mailing, face labels, calendar, holiday cards, renewal, donor campaign = direct mail.
- Golf = little city invitational

In [29]:
#new
events_list = ['general operating budget',
'annual giving',
'teddy bear',
'book sale',
'parent',
'direct mail',
'circle of love',
'newsletter',
'pledge',
'santa stickers',
'smiles campaign',
'holiday mailing',
'verne carson',
'face labels',
'calendar',
'holiday cards',
'renewal',
'donor campaign',
'golf',
'little city invitational',
'university of illinois work place giving',
'residential services',
'ride for smiles',
'poker casino night',
'valentine day',
'budget cut emergency appeal',
'raffle',
'holiday appeal opening doors']

In [30]:
len(events_list)

28

### Iterate over each row in the .csv file
- create a empty list for each row
- if one (or more) of the events in the events_list is found in each row's 'concatenated' column accordingly, append that event to the list (one row can have more than one event/way of donation)

In [31]:
# Iterate through each row of the 'gifttest' DataFrame
for index, row in gifttest.iterrows():
    # Create a list to store matching events for the current row
    matching_events_list = []
    
    # Iterate through each event in 'events_list'
    for event in events_list:
        # Check if the lowercase version of 'concatenated' contains the lowercase version of the current event
        if row['concatenated'].lower().find(event.lower()) != -1:
            # If the event is found, append it to the matching events list
            matching_events_list.append(event)

    # Assign the matching events list to the 'matching_event' column for the current row
    gifttest.at[index, 'matching_event'] = matching_events_list

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_91509/4123412955.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'direct mail' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gifttest.at[index, 'matching_event'] = matching_events_list


In [32]:
gifttest['matching_event'].value_counts()

matching_event
[direct mail]                                  189219
[]                                             110769
[renewal]                                       48111
[general operating budget]                      22463
[annual giving]                                 15502
                                                ...  
[renewal, donor campaign]                           1
[general operating budget, holiday mailing]         1
[budget cut emergency appeal]                       1
[newsletter, residential services]                  1
direct mail                                         1
Name: count, Length: 62, dtype: int64

### Replace empty list [] with NA values

In [33]:
empty_list_rows = gifttest[gifttest['matching_event'].apply(lambda x: x == [])]
import numpy as np

# Replace empty lists with NaN in the 'matching_event' column
gifttest['matching_event'] = gifttest['matching_event'].apply(lambda x: np.nan if x == [] else x)

### Create a subcategory for events called 'newsletter'
- it has a four-season series: 'spring', 'summer', 'fall', 'winter'

In [38]:
# Define a function to check conditions and return the matched season
def get_season(row):
    seasons = ['spring', 'summer', 'fall', 'winter']
    for season in seasons:
        if 'newsletter' in row['concatenated'].lower() and season in row['concatenated'].lower():
            return season

# Apply the function to create a new column 'newsletter_sub'
gifttest['newsletter_sub'] = gifttest.apply(get_season, axis=1)

# Display the updated DataFrame
gifttest['newsletter_sub'].value_counts()

newsletter_sub
fall      9415
winter    6194
spring    5137
summer    4918
Name: count, dtype: int64

### Create a new column for events belonging to 'annual giving' type
* general operating budget, annual giving, teddy bear, book sale, parent = annual giving

In [41]:
annual_giving_events = [
    'general operating budget', 
    'annual giving', 
    'teddy bear', 
    'book sale', 
    'parent'
]
# Make sure 'matching_event' is a list. If it's NaN or None, replace with an empty list
gifttest['matching_event'] = gifttest['matching_event'].apply(lambda x: x if isinstance(x, list) else [])
for index, row in gifttest.iterrows():
    # Check if any event in 'annual_giving_events' is in the 'matching_event' list for the current row
    if any(event in row['matching_event'] for event in annual_giving_events):
        # If a match is found, tag as "annual giving"
        tag = "annual giving"
    else:
        # If no match is found, do not tag
        tag = None
    
    # Assign the tag to the 'tag_annual_giving' column for the current row
    gifttest.at[index, 'tag_annual_giving'] = tag

### Create a new column for events belonging to 'direct mail' type
* direct mail, circle of love, newsletter, pledge, santa stickers, smiles campaign, holiday mailing, verne carson mailing, face labels, calendar, holiday cards, renewal, donor campaign = direct mail

In [43]:
# List of events that should trigger a "direct mail" tag
direct_mail_events = [
    'direct mail', 'circle of love', 'newsletter', 'pledge', 'santa stickers', 
    'smiles campaign', 'holiday mailing', 'verne carson mailing', 'face labels', 
    'calendar', 'holiday cards', 'renewal', 'donor campaign'
]

# Iterate through each row of the 'gifttest' DataFrame
for index, row in gifttest.iterrows():
    # Check if 'matching_event' is not a list, then make it an empty list
    if not isinstance(row['matching_event'], list):
        row['matching_event'] = []

    # Lowercase the elements in 'matching_event' for case-insensitive matching
    matching_events_lower = [event.lower() for event in row['matching_event']]

    # Check if any event in 'direct_mail_events' is in the 'matching_event' list for the current row
    if any(event in matching_events_lower for event in direct_mail_events):
        # If a match is found, tag as "direct mail"
        tag = "direct mail"
    else:
        # If no match is found, do not tag
        tag = None
    
    # Assign the tag to the 'tag_direct_mail' column for the current row
    gifttest.at[index, 'tag_direct_mail'] = tag

# Check the result
gifttest.head()

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_91509/2203755121.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'direct mail' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gifttest.at[index, 'tag_direct_mail'] = tag


,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,...,Appeal Description,Luminate Online Gift ID,NetCommunity Online Gift ID,Gift Code,OPX Provider Description,concatenated,matching_event,newsletter_sub,tag_annual_giving,tag_direct_mail
0,Individual,1026027.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,98 April Renewal,NaN,NaN,NaN,NaN,direct mail,[],None,None,NaN
1,Individual,1041282.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,98 May Renewal,NaN,NaN,NaN,NaN,direct mail,[direct mail],None,None,direct mail
2,Individual,1041328.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,98 March Renewal,NaN,NaN,NaN,NaN,direct mail,[direct mail],None,None,direct mail
3,Individual,1111161.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,98 May Renewal,NaN,NaN,NaN,NaN,direct mail,[direct mail],None,None,direct mail
4,Individual,1113516.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,98 May Renewal,NaN,NaN,NaN,NaN,direct mail,[direct mail],None,None,direct mail


### Create a new column for events belonging to 'golf' type
* golf = little city invitational

In [57]:
# Event that should trigger a "golf" tag
golf_event = ['golf','little city invitational']

# Iterate through each row of the 'gifttest' DataFrame
for index, row in gifttest.iterrows():
    # Ensure 'matching_event' is a list and lowercase elements for case-insensitive matching
    if not isinstance(row['matching_event'], list):
        row['matching_event'] = []
    # Lowercase the elements in 'matching_event' for case-insensitive matching
    matching_events_lower = [event.lower() for event in row['matching_event']]
    
    # Check if the 'golf_event' is in the 'matching_event' list for the current row
    if any(event in matching_events_lower for event in golf_event):
        # If a match is found, tag as "golf"
        tag = "golf"
    else:
        # If no match is found, do not tag
        tag = None
    
    # Assign the tag to the 'tag_golf' column for the current row
    gifttest.at[index, 'tag_golf'] = tag

# Check the result
gifttest.head()

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_91509/2248505986.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'golf' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gifttest.at[index, 'tag_golf'] = tag


,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,...,NetCommunity Online Gift ID,Gift Code,OPX Provider Description,concatenated,matching_event,newsletter_sub,tag_annual_giving,tag_direct_mail,tag_golf,Direct Mail
0,Individual,1026027.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[],None,0,0,NaN,1
1,Individual,1041282.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,0,1,NaN,1
2,Individual,1041328.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,0,1,NaN,1
3,Individual,1111161.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,0,1,NaN,1
4,Individual,1113516.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,0,1,NaN,1


In [72]:
empty_list_count = gifttest['matching_event'].apply(lambda x: x == []).sum()

# Print the count
print("Number of empty lists in 'matching_event':", empty_list_count)

empty_list_count/len(gifttest)*100 

Number of empty lists in 'matching_event': 110770


21.31040625787573

Almost cover **80%** of the data after tagging present events.

# Create Dummy Columns
First, we will create dummy variables for rows that have been tagged and do not have None or NaN values in the 'tag_annual_giving', 'tag_direct_mail', 'tag_golf' columns.

Next, we will remove these tagged rows and create dummy variables for the remaining rows based on the unique events in their 'matching_event' column.

In [58]:
# Function to convert to dummy (1 if value exists, else 0)
convert_to_dummy = lambda x: 0 if pd.isna(x) else 1

# Apply this function to the specified columns
gifttest['tag_annual_giving'] = gifttest['tag_annual_giving'].apply(convert_to_dummy)
gifttest['tag_direct_mail'] = gifttest['tag_direct_mail'].apply(convert_to_dummy)
gifttest['tag_golf'] = gifttest['tag_golf'].apply(convert_to_dummy)

# Check the result
gifttest.head()

,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,...,NetCommunity Online Gift ID,Gift Code,OPX Provider Description,concatenated,matching_event,newsletter_sub,tag_annual_giving,tag_direct_mail,tag_golf,Direct Mail
0,Individual,1026027.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[],None,1,1,0,1
1,Individual,1041282.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,1,1,0,1
2,Individual,1041328.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,1,1,0,1
3,Individual,1111161.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,1,1,0,1
4,Individual,1113516.0,7/1/1998,5.0,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,NaN,NaN,NaN,direct mail,[direct mail],None,1,1,0,1


### Count for three duplicate types

In [59]:
tag_annual_giving_count = gifttest['tag_annual_giving'].sum()
tag_direct_mail_count = gifttest['tag_direct_mail'].sum()
tag_golf_count = gifttest['tag_golf'].sum()
print("Number of rows with 'annual giving' tag:", tag_annual_giving_count)
print("Number of rows with 'direct mail' tag:", tag_direct_mail_count)
print("Number of rows with 'golf' tag:", tag_golf_count)

Number of rows with 'annual giving' tag: 519793
Number of rows with 'direct mail' tag: 519793
Number of rows with 'golf' tag: 9469


# Merge file

In [16]:
# Explode the lists in the 'matching_event' column to separate rows
gifttest_exploded = gifttest['matching_event'].explode('matching_event')

# Create dummy columns for each event
dummy_columns = pd.get_dummies(gifttest_exploded, prefix='event')
dummy_columns = dummy_columns.astype(int)  # or dummy_columns = dummy_columns * 1

# Concatenate the dummy columns with the original dataframe
gifttest = pd.concat([gifttest, dummy_columns], axis=1)

# Display the resulting dataframe
gifttest.head(10)

,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,...,event_newsletter,event_parent,event_pledge,event_renewal,event_santa stickers,event_smiles campaign,event_teaser,event_teddy bear,event_university of illinois work place giving,event_verne carson letter
0,Individual,1026027.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
1,Individual,1041282.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
2,Individual,1041328.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
3,Individual,1111161.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
4,Individual,1113516.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
5,Individual,1033818.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
6,Individual,1027399.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
7,Individual,1043888.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
8,Individual,1094326.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
9,Individual,1106432.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0


In [17]:
gifttest.to_csv('merge_file.csv', index=False)

# One-hot encoding
1 - Yes

0 - No

Direct Mail and Renewal both listed Direct Mail - coding changed

Both together cover all the data. 

### Way of Donation: Direct Mail

In [51]:
gifttest['Direct Mail'] = gifttest['Fund Description'].apply(lambda x: 1 if pd.notna(x) and 'Direct Mail' in x else 0)
gifttest['Direct Mail'].value_counts()

Direct Mail
1    417896
0    101897
Name: count, dtype: int64

### Loyalty: Renewal


In [68]:
gifttest['Renewal'] = gifttest['concatenated'].apply(lambda x: 1 if pd.notna(x) and 'renewal' in x else 0)
gifttest['Renewal'].value_counts()

Renewal
0    464413
1     55380
Name: count, dtype: int64

### Format the 'Gift Amount' column

In [52]:
gifttest['Gift Amount'] = gifttest['Gift Amount'].str.replace('$', '').str.replace(',', '').str.strip()

In [53]:
gifttest['Gift Amount'] = pd.to_numeric(gifttest['Gift Amount'], errors='coerce')

### Total amount of donation in the file

In [54]:
total_gift_amount = gifttest['Gift Amount'].sum()
print(total_gift_amount)

110648616.72


# Mapping

In [60]:
event_to_category = {
    'general operating budget': 'annual giving',
    'annual giving': 'annual giving',
    'teddy bear': 'annual giving',
    'book sale': 'annual giving',
    'parent': 'annual giving',
    'direct mail': 'direct mail',
    'circle of love': 'direct mail',
    'newsletter': 'direct mail',
    'pledge': 'direct mail',
    'santa stickers': 'direct mail',
    'smiles campaign': 'direct mail',
    'holiday mailing': 'direct mail',
    'verne carson mailing': 'direct mail',
    'face labels': 'direct mail',
    'calendar': 'direct mail',
    'holiday cards': 'direct mail',
    'renewal': 'direct mail',
    'donor campaign': 'direct mail',
    'little city invitational': 'golf'  
}

# Sum over giftamount for each event category
- iterate over row
- find the matched event in the 'matching_event' column
- calculate the cumulative sum of each event and round to integer
- save into a dictionary called sumdict

In [63]:
# Initialize a dictionary to hold the sum of Gift Amounts for each category
category_sums = {}

for index, row in gifttest.iterrows():
    matching_events = row['matching_event']

    # Check if matching_events is iterable, if not, make it a list
    if not isinstance(matching_events, (list, tuple)):
        matching_events = [matching_events]

    # Set to keep track of categories already counted for this row
    counted_categories = set()

    for event in matching_events:
        # Map the event to its category
        category = event_to_category.get(event, event)  # Default to the event itself if not found in mapping

        # Check if the category has already been added for this row
        if category not in counted_categories:
            counted_categories.add(category)
            category_sums[category] = category_sums.get(category, 0) + row['Gift Amount']

# Sort the sums by descending order
sorted_category_sums = dict(sorted(category_sums.items(), key=lambda x: x[1], reverse=True))

rounded_sumdict = {key: round(value, 0) for key, value in sorted_category_sums.items()}
rounded_sumdict

{'direct mail': 17441436.0,
 'annual giving': 10441902.0,
 'golf': 9537205.0,
 'residential services': 441340.0,
 'raffle': 196481.0,
 'holiday appeal opening doors': 180396.0,
 'ride for smiles': 113292.0,
 'verne carson': 99202.0,
 'budget cut emergency appeal': 93340.0,
 'university of illinois work place giving': 3889.0}

### Percentage of donation through 'direct mail'

In [64]:
round(rounded_sumdict['direct mail'] / total_gift_amount *100,2)

15.76

### Percentage of donation through 'annual giving'

In [65]:
round(rounded_sumdict['annual giving'] / total_gift_amount * 100,2)

9.44

### Percentage of donation through 'golf'

In [67]:
round(rounded_sumdict['golf'] / total_gift_amount * 100,2)

8.62

### Save the file for dashboard preparation

In [66]:
df = pd.DataFrame(list(rounded_sumdict.items()), columns=['Event', 'Rounded Value'])
df.to_csv('sumdict.csv')

# Count over each event category
- iterate over row
- find the matched event in the 'matching_event' column
- calculate the number of apperance of each event
- save into a dictionary called countdict

In [73]:
#new
countdict = {}

for index, row in gifttest.iterrows():
    matching_events = row['matching_event']
    
    # Check if matching_events is iterable, if not, make it a list
    if not isinstance(matching_events, (list, tuple)):
        matching_events = [matching_events]

    # Set to keep track of categories already counted for this row
    counted_categories = set()

    for event in matching_events:
        # Map the event to its category
        category = event_to_category.get(event, event)  # Default to the event itself if not found in mapping

        # Check if the category has already been counted for this row
        if category not in counted_categories:
            counted_categories.add(category)
            countdict[category] = countdict.get(category, 0) + 1

# Sort the dictionary by count in descending order
sorted_countdict = dict(sorted(countdict.items(), key=lambda x: x[1], reverse=True))

# Display the result
sorted_countdict

{'direct mail': 341300,
 'annual giving': 98413,
 'golf': 9469,
 'raffle': 2877,
 'verne carson': 2265,
 'university of illinois work place giving': 2131,
 'residential services': 1768,
 'budget cut emergency appeal': 1668,
 'ride for smiles': 1646,
 'holiday appeal opening doors': 1364}

In [38]:
#old
countdict = {}

for index, row in gifttest.iterrows():
    matching_events = row['matching_event']
    
    # Check if matching_events is iterable
    if not isinstance(matching_events, (list, tuple)):
        matching_events = [matching_events]

    for event in matching_events:
        if event not in countdict:
            countdict[event] = 1
        else:
            countdict[event] += 1

sorted_countdict = dict(sorted(countdict.items(), key=lambda x: x[1], reverse=True))

# Round the final result to 0 decimal places
rounded_countdict = {key: round(value, 0) for key, value in sorted_countdict.items()}

rounded_countdict

{'direct mail': 211317,
 nan: 186697,
 'renewal': 55380,
 'general operating budget': 43035,
 'newsletter': 41724,
 'annual giving': 41202,
 'circle of love': 26582,
 'pledge': 10847,
 'parent': 8608,
 'golf': 8434,
 'annual dinner': 5561,
 'little city invitational': 5494,
 'santa stickers': 5026,
 'holiday mailing': 4727,
 'teaser': 4500,
 'teddy bear': 4183,
 'calendar': 2814,
 'donor campaign': 2420,
 'verne carson letter': 2265,
 'university of illinois work place giving': 2131,
 'book sale': 2123,
 'holiday cards': 1104,
 'face labels': 154,
 'smiles campaign': 25}

### Save the file for dashboard preparation

In [75]:
df = pd.DataFrame(list(sorted_countdict.items()), columns=['Event', 'Count'])
df.to_csv('countdict.csv')

# Future discussion
- may drop NAN when presenting the results in dashboard (because we didn't cover all the events yet)

-> how to deal with much NAs?
- may use eyeballs to catch more events to decrease the number of NA values